In [6]:
# !pip install mediapipe opencv-python pandas scikit-learn pickle

In [1]:
import mediapipe as mp
import cv2
import csv
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import pickle

In [2]:
import tensorflow as tf
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'

### Importing our Models

In [39]:
# a list of models with predict_proba parameter 
from sklearn.utils import all_estimators

estimators = all_estimators()

for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)

AdaBoostClassifier
BaggingClassifier
BayesianGaussianMixture
BernoulliNB
CalibratedClassifierCV
CategoricalNB
ClassifierChain
ComplementNB
DecisionTreeClassifier
DummyClassifier
ExtraTreeClassifier
ExtraTreesClassifier
GaussianMixture
GaussianNB
GaussianProcessClassifier
GradientBoostingClassifier
GridSearchCV
HalvingGridSearchCV
HalvingRandomSearchCV
HistGradientBoostingClassifier
KNeighborsClassifier
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LogisticRegression
LogisticRegressionCV
MLPClassifier
MultiOutputClassifier
MultinomialNB
NuSVC
OneVsRestClassifier
Pipeline
QuadraticDiscriminantAnalysis
RFE
RFECV
RadiusNeighborsClassifier
RandomForestClassifier
RandomizedSearchCV
SGDClassifier
SVC
SelfTrainingClassifier
StackingClassifier
VotingClassifier


In [3]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier, BaggingClassifier

In [4]:
# Mediapipe Drawing utilities and the Holistic Model
mp_drawing  = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## Detection on videos

In [48]:
# capturing the video from web cam
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    # looping on each video frame
    while cap.isOpened():
        ret , frame = cap.read()
        # Re-colour the frame captured from BGR to RGB 
        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        
        # Re-colour the frame captured from RGB to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) , mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))
        
        # Showing results on the screen
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Landmarks in our holistic model
<img src="https://i.imgur.com/AzKNp7A.png"> 

## Capturing the needed landamarks & Exporting to CSV

In [41]:
# Capturing face and pose landmarks 
face_landmarks = len(results.face_landmarks.landmark)
pose_landmarks = len(results.pose_landmarks.landmark)
num_coordinates = face_landmarks + pose_landmarks
num_coordinates

501

In [42]:
# class: for class category, x, y, z: for the coordinates, and v: for the visibility (1 or 0)
landmarks = ['class']
for val in range(1,num_coordinates+1):
    landmarks += ['x{}'.format(val) ,'y{}'.format(val),'z{}'.format(val),'v{}'.format(val) ]

In [43]:
landmarks[-1]

'v501'

In [44]:
# Writing the columns in our csv
with open('coords.csv' , mode='w',newline='') as f:
    csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Class names

In [29]:
# class_name = 'Cheerful'
# class_name = 'Happy'
# class_name = 'Sad'
class_name = 'OK'
# class_name = 'Not_OK'
# class_name = 'Love'

# -----------------------------------------------------
# Additional features 

# class_name = 'Stop'
# class_name = 'Shocked'
# class_name = 'Peace'
# class_name = 'Wait'
# class_name = 'Cant hear'
# class_name = 'Angry'

In [30]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) , mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) , mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))

        # export coordinates
        try:
            # Extracting pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in pose]).flatten())
            
            # Extracting face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in face]).flatten())
            
            # Combining pose and face 
            row = pose_row + face_row
            # Appending class name 
            row.insert(0 , class_name)
            
            # Writing the coordinates to our csv with the specified class 
            with open('coords1.csv' , mode='a',newline='') as f:
                csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Create our Model

In [31]:
df = pd.read_csv('coords1.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.544411,0.484712,-1.273281,0.999684,0.579251,0.405091,-1.217857,0.999368,0.598930,...,-0.013160,0.0,0.629015,0.416093,0.009674,0.0,0.635546,0.411834,0.010136,0.0
1,Happy,0.543680,0.485149,-1.245793,0.999709,0.578499,0.405450,-1.196947,0.999415,0.597703,...,-0.011936,0.0,0.625433,0.410008,0.010990,0.0,0.631891,0.405959,0.011462,0.0
2,Happy,0.543305,0.485228,-1.301247,0.999722,0.578102,0.405530,-1.252712,0.999440,0.597062,...,-0.012390,0.0,0.623901,0.410824,0.010665,0.0,0.630346,0.406485,0.011180,0.0
3,Happy,0.542873,0.485261,-1.338242,0.999738,0.577738,0.405641,-1.286133,0.999469,0.596567,...,-0.012002,0.0,0.624709,0.408597,0.011734,0.0,0.630903,0.405141,0.012228,0.0
4,Happy,0.543544,0.487443,-1.500913,0.999751,0.578165,0.406435,-1.444013,0.999492,0.596890,...,-0.012166,0.0,0.625862,0.410949,0.011609,0.0,0.632130,0.406974,0.012178,0.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4255 entries, 0 to 4254
Columns: 2005 entries, class to v501
dtypes: float64(2004), object(1)
memory usage: 65.1+ MB


In [33]:
# X: our feature values, y: our classes
X = df.drop('class', axis=1)
y = df['class']

In [34]:
y.unique()

array(['Happy', 'Cheerful', 'Sad', 'Love', 'OK', 'Not_OK'], dtype=object)

In [35]:
# Splitting the data into training and testing sets with ratio (80:20)
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state=42)

In [36]:
len(X_train) , len(X_test)

(3404, 851)

### Training different models

In [37]:
# making a pipeline for each classifier so the data is standardized then trained on the classifier
pipelines = {
    'lr':make_pipeline(StandardScaler() , LogisticRegression()),
    'rf':make_pipeline(StandardScaler() , RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler() , GradientBoostingClassifier()),
    'lr_cv':make_pipeline(StandardScaler() , LogisticRegressionCV()),
    'svm':make_pipeline(StandardScaler() , SVC()),
    'sgd':make_pipeline(StandardScaler() , SGDClassifier()),
    'bc':make_pipeline(StandardScaler() , BaggingClassifier())
}

In [38]:
fit_models = {}
for algo , pipeline in pipelines.items():
    model = pipeline.fit(X_train , y_train)
    fit_models[algo] = model

D:\downloads\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\downloads\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = 

## Evaluating the Accuracy for our Models

In [39]:
for algo , model in fit_models.items():
    pred = model.predict(X_test.values)
    print(algo , accuracy_score(y_test , pred))

D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


lr 1.0
rf 1.0
gb 1.0
lr_cv 1.0
svm 1.0
sgd 1.0
bc 1.0


D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Saving and Serializing our best Model

In [40]:
with open('body_language3.pkl' , 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## Make Predictions on the best Model in Real-Time

In [41]:
with open('body_language3.pkl','rb') as f:
    model = pickle.load(f)

In [42]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [43]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.   0.5  0.12 0.1  0.27 0.01]
Happy [0.   0.48 0.08 0.11 0.33 0.  ]


D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.   0.52 0.11 0.08 0.28 0.01]
Happy [0.   0.45 0.06 0.17 0.31 0.01]


D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Sad [0.02 0.02 0.04 0.01 0.02 0.89]
Sad [0.02 0.02 0.02 0.01 0.02 0.91]


D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.01 0.02 0.02 0.01 0.02 0.92]
Sad [0.01 0.02 0.07 0.05 0.01 0.84]


D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
D:\downloads\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.02 0.05 0.07 0.03 0.01 0.82]
Sad [0.02 0.09 0.07 0.05 0.   0.77]
